* [hopsworks-tutorials churn_feature_pipeline](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/churn/1_churn_feature_pipeline.ipynb)

In [2]:
!pip3 install -U hopsworks --quiet

In [12]:
import pandas as pd

#ignore warnings
import warnings, requests, io
warnings.filterwarnings('ignore')

In [17]:
# Read demography data
demography_url ="https://repo.hops.works/dev/davit/churn/demography.csv"
#demography_df = pd.read_csv(demography_url)
x = requests.get(url=demography_url).content 
demography_df = pd.read_csv(io.StringIO(x.decode('utf8')))
demography_df.head()

,customerID,gender,SeniorCitizen,Dependents,Partner
0,7590-VHVEG,Female,0,No,Yes
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [20]:
customer_info_url =  "https://repo.hops.works/dev/davit/churn/customer_info.csv"

x2 = requests.get(url=customer_info_url).content 
customer_info_df = pd.read_csv(io.StringIO(x2.decode('utf8')), parse_dates=['datetime'])
customer_info_df.head(3)

,customerID,Contract,tenure,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,datetime
0,7590-VHVEG,Month-to-month,1,Electronic check,Yes,29.85,29.85,No,2021-10-25 15:07:18.625390512
1,5575-GNVDE,One year,34,Mailed check,No,56.95,1889.5,No,2020-06-28 06:32:24.674808292
2,3668-QPYBK,Month-to-month,2,Mailed check,Yes,53.85,108.15,Yes,2021-12-05 20:10:58.449304176


In [23]:
# Read subscriptions data with datetime parsing
subscriptions_url = "https://repo.hops.works/dev/davit/churn/subscriptions.csv"
x3 = requests.get(url=subscriptions_url).content
subscriptions_df = pd.read_csv(io.StringIO(x3.decode('utf8')), parse_dates=['datetime'])
subscriptions_df .head(3)

,customerID,DeviceProtection,OnlineBackup,OnlineSecurity,InternetService,MultipleLines,PhoneService,TechSupport,StreamingMovies,StreamingTV,datetime
0,7590-VHVEG,No,Yes,No,DSL,No phone service,No,No,No,No,2021-10-25 15:07:18.625390512
1,5575-GNVDE,Yes,No,Yes,DSL,No,Yes,No,No,No,2020-06-28 06:32:24.674808292
2,3668-QPYBK,No,Yes,Yes,DSL,No,Yes,No,No,No,2021-12-05 20:10:58.449304176


### Feature engineering

In [24]:
# Convert the "TotalCharges" column to numeric, treating errors as NaN
customer_info_df["TotalCharges"] = pd.to_numeric(
    customer_info_df["TotalCharges"], 
    errors='coerce',
)

# Replace NaN values in the "TotalCharges" column with 0
customer_info_df["TotalCharges"].fillna(0, inplace=True)

# Replace values in the "Churn" column with 0 for "No" and 1 for "Yes"
customer_info_df["Churn"].replace({"No": 0, "Yes": 1}, inplace=True)

### Creating Feature Groups

In [28]:
!pip3 install hsfs

In [30]:
import hsfs

ModuleNotFoundError: No module named 'distutils'

In [40]:
!pip3 install -U hopsworks    
!pip3 install hsfs    


In [42]:
import joblib
from xgboost import plot_importance
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [43]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

ModuleNotFoundError: No module named 'distutils'